In [1]:
import dotenv
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel

dotenv.load_dotenv()

print("Loading tokenizer and model...")
base_model = AutoModelForSequenceClassification.from_pretrained(
    "unsloth/Meta-Llama-3.1-8B",
    device_map="auto",
    num_labels=1,
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B")

print("Loading PEFT model...")
peft_model = PeftModel.from_pretrained(
    base_model, "./reward_model_output/checkpoint-30000/", device_map="auto"
)

print("Merging PEFT model with base model...")
merged_model = peft_model.merge_and_unload()

print("Saving merged model...")
merged_model.save_pretrained("./models/llama_32_8b_merged")
tokenizer.save_pretrained("./models/llama_32_8b_merged")

Loading tokenizer and model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/Meta-Llama-3.1-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading PEFT model...
Merging PEFT model with base model...
Saving merged model...


('./models/llama_32_8b_merged/tokenizer_config.json',
 './models/llama_32_8b_merged/special_tokens_map.json',
 './models/llama_32_8b_merged/tokenizer.json')

In [3]:
from dotenv import load_dotenv

load_dotenv()

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    "unsloth/Meta-Llama-3.1-8B", device_map="auto", num_labels=1
)

model = AutoModelForSequenceClassification.from_pretrained(
    "./models/stories_train_model_v3", device_map="auto", num_labels=1
)
tokenizer = AutoTokenizer.from_pretrained("./models/stories_train_model_v3")

model = model.merge_and_unload()

from huggingface_hub import HfApi

# Login to Hugging Face (make sure you have your token set in the environment)
api = HfApi()

# Create private repository and push model
model.push_to_hub("OpenPipe/lat-pretrained-rm-v1", private=True, use_auth_token=True)

# Push tokenizer
tokenizer.push_to_hub(
    "OpenPipe/lat-pretrained-rm-v1", private=True, use_auth_token=True
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/Meta-Llama-3.1-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for LlamaForSequenceClassification:
	size mismatch for score.modules_to_save.default.weight: copying a param with shape torch.Size([1, 4096]) from checkpoint, the shape in current model is torch.Size([2, 4096]).